## Prepare Geojson Data

In [73]:
import pandas as pd
import geojson

In [74]:
GEOJSON_PATH = 'datakind_housing_data/cleaned/county.geojson'

In [75]:
chas_data = pd.read_csv('datakind_housing_data/cleaned/chas_cost_burden.csv')

In [76]:
chas_data[(chas_data['st'] == 1) & (chas_data['cnty'] == 3)]

,name,st,cnty,T8_est7,T8_est20,T8_est33,T8_est46,T8_est59,T8_est10,T8_est23,...,owned_no_record,rented_no_record,owned_population,rented_population,owned_total_burdend,rented_total_burdend,owned_burdend_percentile,rented_burdend_percentile,owned_severe_burdend_percentile,rented_severe_burdend_percentile
1,"Baldwin County, Alabama",1,3,945,1085,2710,835,1430,3410,1265,...,395,140,66845,19810,12375,7590,0.18513,0.38314,0.080335,0.172892


In [77]:
with open(GEOJSON_PATH) as f:
    conty_gj = geojson.load(f)

In [78]:
conty_gj['features'][0]['properties'].keys()

dict_keys(['GEOID', 'FIPS_CODE', 'COUNTY_STATE_CODE', 'COUNTY_STATE_NAME'])

In [79]:
counter = 0
for i in range(len(conty_gj['features'])):
    st = int(conty_gj['features'][i]['properties']['FIPS_CODE'][:2])
    cty = int(conty_gj['features'][i]['properties']['FIPS_CODE'][-3:])
    try:
        county_data = chas_data[(chas_data['st'] == st) & (chas_data['cnty'] == cty)].iloc[0]
        conty_gj['features'][i]['properties']['renter_population'] = str(county_data['rented_population'])
        conty_gj['features'][i]['properties']['renter_burdend'] = str(county_data['rented_burdend_percentile'])
        conty_gj['features'][i]['properties']['renter_severe_burdend'] = str(county_data['rented_severe_burdend_percentile'])
        counter += 1
    except Exception as e:
        print(conty_gj['features'][i]['properties'])
        
counter

{'GEOID': '78010', 'FIPS_CODE': '78-010', 'COUNTY_STATE_CODE': 'St. Croix, undefined', 'COUNTY_STATE_NAME': 'St. Croix, undefined'}
{'GEOID': '78020', 'FIPS_CODE': '78-020', 'COUNTY_STATE_CODE': 'St. John, undefined', 'COUNTY_STATE_NAME': 'St. John, undefined'}
{'GEOID': '78030', 'FIPS_CODE': '78-030', 'COUNTY_STATE_CODE': 'St. Thomas, undefined', 'COUNTY_STATE_NAME': 'St. Thomas, undefined'}
{'GEOID': '02261', 'FIPS_CODE': '02-261', 'COUNTY_STATE_CODE': 'Valdez-Cordova, AK', 'COUNTY_STATE_NAME': 'Valdez-Cordova, Alaska'}


3213

In [80]:
chas_data[(chas_data['st'] == 2)]['cnty'].unique()

array([ 13,  16,  20,  50,  60,  63,  66,  68,  70,  90, 100, 105, 110,
       122, 130, 150, 158, 164, 170, 180, 185, 188, 195, 198, 220, 230,
       240, 275, 282, 290])

In [81]:
chas_data['st'].max()

np.int64(72)

In [84]:
with open('datakind_housing_data/curated/county_renter_affordablity.geojson', 'w') as f:
   geojson.dump(conty_gj, f)